In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [11]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(500, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66)]
  wgt = [(0.4, 0.4, 0.5, 0.55, 0.85)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312      1.0 -4.319705e-04                  NaN           NaN
0.000312      NaN           NaN                  1.0 -2.089382e-05
0.000750      2.0 -4.312287e-04                  2.0 -2.540344e-05
0.001052      3.0 -4.292894e-04                  3.0 -1.640265e-05
0.001476      4.0 -4.271620e-04                  4.0 -1.489382e-05
0.002070      5.0 -4.248950e-04                  5.0 -1.319812e-05
0.002904      6.0 -4.223493e-04                  6.0 -1.143252e-05
0.004074      7.0 -4.196678e-04                  7.0 -9.772353e-06
0.005714      8.0 -4.169421e-04                  8.0 -7.799030e-06
0.008015      9.0 -4.141060e-04                  9.0 -5.927720e-06
0.011243     10.0 -4.114446e-04                 10.0 -3.860736e-06
0.015771     11.0 -4.088781e-04                 11.0 -1.744709e-06
0.022122     12.0 -4.067335e-04                 12.0  4.456424e-07
0.031031     13.0 -4.051619e-04                 13.0  2.664653e-06
0.043528     14.0 -4.032118e-04                 14.0  4.557346e-06
0.061057     15.0 -3.828951e-04                 15.0  1.651799e-05
0.085645     16.0 -3.151524e-04                 16.0  5.095084e-05
0.120136     17.0 -1.849579e-04                 17.0  1.112356e-04
0.168516     18.0  2.023407e-05                 18.0  2.034469e-04
0.236378     19.0  3.292719e-04                 19.0  3.420866e-04
0.331549     20.0  7.813866e-04                 20.0  5.474991e-04
0.465100     21.0  1.430542e-03                 21.0  8.478256e-04
0.652400     22.0  2.243104e-03                 22.0  1.233760e-03
0.915100     23.0  2.517170e-03                 23.0  1.394114e-03
1.283650     24.0  2.106559e-03                 24.0  1.243818e-03
1.800600     25.0  1.552071e-03                 25.0  1.007095e-03
2.525700     26.0  9.953958e-04                 26.0  7.443348e-04
3.542800     27.0  5.938715e-04                 27.0  5.306436e-04
4.969550     28.0  3.400874e-04                 28.0  3.707254e-04
6.970850     29.0  1.868379e-04                 29.0  2.520442e-04
9.778100     30.0  9.788299e-05                 30.0  1.639242e-04
13.715850    31.0  4.735139e-05                 31.0  9.861821e-05
19.239350    32.0  1.810607e-05                 32.0  5.004814e-05
26.987250    33.0 -2.586253e-07                 33.0  1.356114e-05
37.855300    34.0 -1.430668e-05                 34.0 -1.503777e-05
53.100050    35.0 -3.557504e-05                 35.0 -4.237245e-05
73.887500    36.0 -5.622243e-05                 36.0 -6.364013e-05
97.662500    37.0 -6.824655e-05                 37.0 -7.509354e-05
121.437500   38.0 -7.354455e-05                 38.0 -8.131661e-05
145.212500   39.0 -7.071632e-05                 39.0 -8.256327e-05
168.987500   40.0 -6.617717e-05                 40.0 -8.304617e-05
192.762500   41.0 -6.736176e-05                 41.0 -9.588297e-05
216.537500   42.0 -8.929885e-05                 42.0 -1.514346e-04
240.312500   43.0 -9.870020e-05                 43.0 -2.325606e-04
264.087500   44.0 -9.103736e-05                 44.0 -3.835767e-04
287.862500   45.0 -5.632962e-05                 45.0 -4.748709e-04
311.637500   46.0  2.286311e-05                 46.0 -3.644250e-04
335.412500   47.0  1.609540e-04                 47.0 -1.759828e-04
359.187500   48.0  3.599475e-04                 48.0 -1.849238e-04
382.962500   49.0  6.592809e-04                 49.0 -4.294443e-04
406.737500   50.0  1.150038e-03                 50.0 -5.858602e-04
430.512500   51.0  1.830619e-03                 51.0 -3.128986e-04
454.287500   52.0  2.713052e-03                 52.0  6.087314e-04
478.062500   53.0  3.809038e-03                 53.0  2.176110e-03
501.837500   54.0  5.124194e-03                 54.0  4.340393e-03
525.612500   55.0  6.675529e-03                 55.0  6.839466e-03
549.387500   56.0  8.459949e-03                 56.0  9.47523

# Fluxes by Layer

CRD                            CLIRAD            \
                        flug          fldg     fnetg      flug      fldg   
pressure     level                                                         
0.000000e+00 1     -5.975937  0.000000e+00 -5.975937       NaN       NaN   
1.000000e-08 1           NaN           NaN       NaN -5.605172  0.000002   
6.244000e-04 2     -5.975937  4.658226e-09 -5.975937 -5.605172  0.000002   
8.759000e-04 3     -5.975937  6.574360e-09 -5.975937 -5.605172  0.000002   
1.228600e-03 4     -5.975937  9.378620e-09 -5.975937 -5.605172  0.000002   
1.723400e-03 5     -5.975937  1.348593e-08 -5.975937 -5.605172  0.000002   
2.417400e-03 6     -5.975937  1.949963e-08 -5.975937 -5.605172  0.000002   
3.390900e-03 7     -5.975937  2.830448e-08 -5.975937 -5.605172  0.000002   
4.756500e-03 8     -5.975937  4.119427e-08 -5.975937 -5.605172  0.000002   
6.672000e-03 9     -5.975938  6.003845e-08 -5.975938 -5.605172  0.000002   
9.358900e-03 10    -5.975938  8.759352e-08 -5.975938 -5.605172  0.000002   
1.312780e-02 11    -5.975938  1.278486e-07 -5.975938 -5.605172  0.000002   
1.841450e-02 12    -5.975938  1.865784e-07 -5.975938 -5.605172  0.000002   
2.583020e-02 13    -5.975939  2.721806e-07 -5.975938 -5.605172  0.000002   
3.623230e-02 14    -5.975939  3.967594e-07 -5.975939 -5.605172  0.000002   
5.082340e-02 15    -5.975940  5.798636e-07 -5.975940 -5.605172  0.000002   
7.129060e-02 16    -5.975942  8.870841e-07 -5.975941 -5.605172  0.000002   
1.000000e-01 17    -5.975943  1.501470e-06 -5.975942 -5.605172  0.000003   
1.402710e-01 18    -5.975945  2.788380e-06 -5.975943 -5.605172  0.000003   
1.967600e-01 19    -5.975948  5.444128e-06 -5.975943 -5.605172  0.000004   
2.759970e-01 20    -5.975950  1.083457e-05 -5.975939 -5.605170  0.000006   
3.871000e-01 21    -5.975951  2.158755e-05 -5.975929 -5.605167  0.000010   
5.431000e-01 22    -5.975945  4.268558e-05 -5.975903 -5.605160  0.000019   
7.617000e-01 23    -5.975926  8.177503e-05 -5.975845 -5.605146  0.000036   
1.068500e+00 24    -5.975891  1.382140e-04 -5.975753 -5.605122  0.000063   
1.498800e+00 25    -5.975847  2.008989e-04 -5.975646 -5.605093  0.000098   
2.102400e+00 26    -5.975796  2.615357e-04 -5.975535 -5.605060  0.000137   
2.949000e+00 27    -5.975746  3.112505e-04 -5.975435 -5.605027  0.000178   
4.136600e+00 28    -5.975699  3.472095e-04 -5.975351 -5.604995  0.000221   
5.802500e+00 29    -5.975654  3.699254e-04 -5.975284 -5.604965  0.000264   
8.139200e+00 30    -5.975614  3.811638e-04 -5.975232 -5.604939  0.000308   
1.141700e+01 31    -5.975578  3.835961e-04 -5.975194 -5.604919  0.000352   
1.601470e+01 32    -5.975549  3.802985e-04 -5.975169 -5.604908  0.000394   
2.246400e+01 33    -5.975529  3.742491e-04 -5.975155 -5.604910  0.000434   
3.151050e+01 34    -5.975523  3.678629e-04 -5.975155 -5.604932  0.000471   
4.420010e+01 35    -5.975539  3.619276e-04 -5.975177 -5.604984  0.000501   
6.200000e+01 36    -5.975600  3.486464e-04 -5.975252 -5.605089  0.000516   
8.577500e+01 37    -5.975737  3.266875e-04 -5.975410 -5.605262  0.000510   
1.095500e+02 38    -5.975916  3.133188e-04 -5.975602 -5.605461  0.000497   
1.333250e+02 39    -5.976133  3.240816e-04 -5.975809 -5.605684  0.000492   
1.571000e+02 40    -5.976388  3.796287e-04 -5.976008 -5.605938  0.000513   
1.808750e+02 41    -5.976695  5.000453e-04 -5.976195 -5.606239  0.000580   
2.046500e+02 42    -5.977156  7.710858e-04 -5.976385 -5.606693  0.000763   
2.284250e+02 43    -5.978173  1.536701e-03 -5.976636 -5.607722  0.001366   
2.522000e+02 44    -5.980131  3.217359e-03 -5.976914 -5.609746  0.002735   
2.759750e+02 45    -5.983536  6.365731e-03 -5.977171 -5.613325  0.005234   
2.997500e+02 46    -5.989003  1.167387e-02 -5.977329 -5.619141  0.009712   
3.235250e+02 47    -5.997246  1.998124e-02 -5.977265 -5.627975  0.017520   
3.473000e+02 48    -6.009073  3.226177e-02 -5.976811 -5.640796  0.029845   
3.710750e+02 49    -6.025207  4.940949e-02 -5.975798 -5.658472  0.047000 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')